In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
os.chdir('/content/gdrive/MyDrive/book model datas/practice projects/')

In [ ]:
book_details=pd.read_csv('model_2_data.csv',sep=',',error_bad_lines=False,encoding='latin1')

In [ ]:
book1=book_details.fillna(0)

In [ ]:
book1.head()

,user_id,title,author,publisher,isbn,language,no_of_exchanges,rating
0,Z4gCnjTUahoz2qaZmk9GJt,Basics of the U.S. Health Care System: Edition 2,Nancy J. Niles,Jones & Bartlett Learning,9.781284e+12,English,3,2.3
1,PwS6wrs4jTJTRrENQ4ErfH,Financial Accounting Theory,Craig Deegan,McGraw-Hill Education Australia,9.781743e+12,English,7,4.7
2,JysYJh9MJsgB92MH86syxU,Delivering Health Care in America: A Systems A...,Leiyu Shi,Jones & Bartlett Learning,9.781284e+12,English,7,1.5
3,ZNanmaD93chZqkq5JHzM2t,Fraud and Fraud Detection: A Data Analytics Ap...,Sunder Gee,John Wiley & Sons,9.781119e+12,English,1,5.0
4,JGUfL4yHZzQhKSLPcMbc3e,Financial Institutions| Instruments and Market...,Christopher Viney,McGraw-Hill Education Australia,9.781744e+12,English,3,1.7


In [ ]:
book1.shape

(2151, 8)

In [ ]:
book1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2151 entries, 0 to 2150
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           2151 non-null   object 
 1    title            2151 non-null   object 
 2    author           2151 non-null   object 
 3    publisher        2151 non-null   object 
 4    isbn             2151 non-null   float64
 5    language         2151 non-null   object 
 6    no_of_exchanges  2151 non-null   int64  
 7    rating           2151 non-null   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 134.6+ KB


## Content based recommendation system

In [ ]:
# Reverse mapping of indices and movie titles
indices = pd.Series(book1.index, index=book1[' title']).drop_duplicates()

In [ ]:
indices

 title
Basics of the U.S. Health Care System: Edition 2                       0
Financial Accounting Theory                                            1
Delivering Health Care in America: A Systems Approach| Edition 6       2
Fraud and Fraud Detection: A Data Analytics Approach                   3
Financial Institutions| Instruments and Markets| 8th edition           4
                                                                    ... 
Captains of the Sands                                               2146
4 años a bordo de mí mismo: (Diario de los 5 sentidos)              2147
Eres buena y lo sabes                                               2148
Tu cruz en el cielo desierto                                        2149
The World in Half                                                   2150
Length: 2151, dtype: int64

In [ ]:
book1.columns

Index(['user_id', ' title', ' author', ' publisher', ' isbn', ' language',
       ' no_of_exchanges', ' rating'],
      dtype='object')

In [ ]:
book1.sort_values(' rating',ascending=False).head()

,user_id,title,author,publisher,isbn,language,no_of_exchanges,rating
1600,kpGP3V58AnZRZBEpNp3r68,St Piran's: Tiny Miracle Twins (Mills & Boon M...,Maggie Kingsley,HarperCollins UK,9.781409e+12,English,4,5.0
1356,NZS5rpufoHV5pkM4ivfPRn,Ki: The Complete Series,Odette C. Bell,Odette C. Bell,0.000000e+00,English,0,5.0
2014,kxVBUghvxKx94WMn4n3K7n,The Royal Wedding Collection: Once Upon A Prin...,Rachel Hauck,Zondervan,9.780310e+12,English,6,5.0
1453,ekcQ3UUyyBCaVVR2necTZZ,BEDHUND,AVINASH HAMBIRRAO LONDHE,MEHTA PUBLISHING HOUSE,9.789353e+12,Marathi,1,5.0
120,8fkijfv3QzeX9itocbGWc5,Understanding Sociological Theory for Educatio...,Tania Ferfolja,Cambridge University Press,9.781316e+12,English,6,5.0


In [ ]:
user_details=pd.read_csv('model1_data.csv')

In [ ]:
book1=book1.rename(columns={'user_id':'user_id', ' title':'title', ' author':'author', ' publisher':'publisher', ' isbn':'isbn', ' language':'language',
       ' no_of_exchanges':'no_of_exchanges', ' rating':'rating'})

In [ ]:
book1.columns

Index(['user_id', 'title', 'author', 'publisher', 'isbn', 'language',
       'no_of_exchanges', 'rating'],
      dtype='object')

In [ ]:
combine_book_rating = book1.dropna(axis = 0, subset = ['title'])

book_ratingCount = (combine_book_rating.
     groupby(by = ['title'])['no_of_exchanges'].
     count().
     reset_index())
book_ratingCount.head()

,title,no_of_exchanges
0,#6 Shades of Gray: Dawn of the Twilight- Outbreak,2
1,#9 Shades of Gray: Rise Of The Gorgons,1
2,#MexicanRevolution,1
3,10 Minutes 38 Seconds in this Strange World: S...,2
4,100/40 Cien relatos durante la cuarentena,1


In [ ]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
# rating_with_totalRatingCount.head()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(book_ratingCount['no_of_exchanges'].describe())

count   1726.000
mean       1.246
std        0.630
min        1.000
25%        1.000
50%        1.000
75%        1.000
max        7.000
Name: no_of_exchanges, dtype: float64


In [ ]:
print(book_ratingCount['no_of_exchanges'].quantile(np.arange(.9, 1, .01)))

0.900   2.000
0.910   2.000
0.920   2.000
0.930   2.000
0.940   2.000
0.950   3.000
0.960   3.000
0.970   3.000
0.980   3.000
0.990   4.000
Name: no_of_exchanges, dtype: float64


In [ ]:
popularity_threshold = 2
rating_popular_book = rating_with_totalRatingCount.query('no_of_exchanges_x	>= @popularity_threshold')
rating_popular_book.head()

,user_id,title,author,publisher,isbn,language,no_of_exchanges_x,rating,no_of_exchanges_y
0,Z4gCnjTUahoz2qaZmk9GJt,Basics of the U.S. Health Care System: Edition 2,Nancy J. Niles,Jones & Bartlett Learning,9781284066043.000,English,3,2.300,1
1,PwS6wrs4jTJTRrENQ4ErfH,Financial Accounting Theory,Craig Deegan,McGraw-Hill Education Australia,9781743074626.000,English,7,4.700,1
2,JysYJh9MJsgB92MH86syxU,Delivering Health Care in America: A Systems A...,Leiyu Shi,Jones & Bartlett Learning,9781284066067.000,English,7,1.500,1
4,JGUfL4yHZzQhKSLPcMbc3e,Financial Institutions| Instruments and Market...,Christopher Viney,McGraw-Hill Education Australia,9781743764848.000,English,3,1.700,1
5,96GBUfNMdCkarJrvJG8fmp,The New Trading for a Living: Psychology| Disc...,Alexander Elder,John Wiley & Sons,9781118963678.000,English,2,4.700,1


In [ ]:
rating_popular_book.shape

(1686, 9)

In [ ]:
book1.columns

Index(['user_id', 'title', 'author', 'publisher', 'isbn', 'language',
       'no_of_exchanges', 'rating'],
      dtype='object')

# Implementing KNN

In [ ]:
from scipy.sparse import csr_matrix
book_rating_pivot = book1.pivot(index = 'isbn', columns = 'user_id', values = 'no_of_exchanges').fillna(0)
book_rating_matrix = csr_matrix(book_rating_pivot.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(book_rating_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
query_index = np.random.choice(book_rating_pivot.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(book_rating_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

1216


In [ ]:
book_rating_pivot.index[query_index]

9781617508691.0

In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(book_rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, book_rating_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for 9781617508691.0:

1: 9781473542419.0, with distance of 1.0:
2: 9781473542211.0, with distance of 1.0:
3: 9781473542303.0, with distance of 1.0:
4: 9781473542310.0, with distance of 1.0:
5: 9781473541542.0, with distance of 1.0:


## Title as index

In [ ]:
from scipy.sparse import csr_matrix
book1 = book1.drop_duplicates(['user_id', 'title'])
book_rating_pivot = book1.pivot(index = 'title', columns = 'user_id', values = 'rating').fillna(0)
book_rating_matrix = csr_matrix(book_rating_pivot.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute',p=0.7)
model_knn.fit(book_rating_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=0.7,
                 radius=1.0)

In [ ]:
query_index = np.random.choice(book_rating_pivot.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(book_rating_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

1356


In [ ]:
book_rating_pivot.index[query_index]

"The Kiss Quotient: Goodread's Romance Book of the Year"

In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(book_rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, book_rating_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for The Kiss Quotient: Goodread's Romance Book of the Year:

1: The Art of War, with distance of 1.0:
2: The Art of Company Valuation and Financial Statement Analysis: A Value Investor's Guide with Real-life Case Studies, with distance of 1.0:
3: The Art of Racing in the Rain: A Novel, with distance of 1.0:
4: The Assistant, with distance of 1.0:
5: The Art of Writing About Art: Edition 2, with distance of 1.0:
